# Nhận diện bệnh của cây trồng

## Giới thiệu

Môn học: Nhận dạng

Lớp: 19KHMT

Nhằm giúp người trồng cây xác định được loại bệnh đang có ở trên cây trồng, mình sẽ xây dựng một mô hình mạng neural nhân tạo để nhận diện một trong bốn loại bệnh sau đây:
- **Combinations (multiple diseases)**: cây trồng bị nhiều bệnh cùng lúc.
- **Healthy**: cây trồng khỏe mạnh.
- **Rust**: cây trồng bị bệnh nấm gỉ sắt, thường xuất hiện trên lá.
- **Scab**: cây trồng bị vảy, xuất hiện trên cả lá, thân và quả.

Mô hình được sử dụng sẽ là **Mạng thần kinh tích chập (Convolutional Neural Network - CNN)**.

**Data**: Plant Pathology 2020 - FGVC7.

## Đọc dữ liệu

Đọc thông tin về những file sẽ dùng để huấn luyện (ở trong file train.csv) và thông tin về những file sẽ dùng để kiểm tra (ở trong file test.csv).

In [ ]:
import pandas as pd

pd_train = pd.read_csv('../input/plant-pathology-2020-fgvc7/train.csv')
pd_test  = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')

Chọn kích cỡ ảnh phù hợp, ở đây là 100. Con số càng cao, ảnh càng chi tiết, kết quả học tốt hơn nhưng thời gian huấn luyện lâu hơn.

In [ ]:
img_size = 100

Đọc ảnh huấn luyện

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load training images
train_image = []
for name in pd_train['image_id']:
    img   = cv2.imread('../input/plant-pathology-2020-fgvc7/images/' + name + '.jpg')
    image = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_AREA)
    train_image.append(image)

# Display some training images
fig, ax = plt.subplots(1, 4, figsize = (15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(train_image[i])

Đọc ảnh kiểm tra

In [ ]:
# Load test images
test_image = []
for name in pd_test['image_id']:
    img   = cv2.imread('../input/plant-pathology-2020-fgvc7/images/' + name + '.jpg')
    image = cv2.resize(img, (img_size, img_size), interpolation = cv2.INTER_AREA)
    test_image.append(image)

# Display some test images
fig, ax = plt.subplots(1, 4, figsize = (15, 15))
for i in range(4):
    ax[i].set_axis_off()
    ax[i].imshow(test_image[i])  

## Chuẩn bị dữ liệu

Feature

In [ ]:
feature = np.ndarray(shape = (len(train_image), img_size, img_size, 3),
                     dtype = np.float32)

for i in range(len(train_image)):
    feature[i] = train_image[i]

feature = feature / 255

Label

In [ ]:
label = pd_train.copy()
del label['image_id']

label = np.array(label.values)

Chia dữ liệu thành dữ liệu huấn luyện thành hai phần, training và validation

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(feature, label, test_size = 0.2, random_state = 42)

Test

In [ ]:
x_test = np.ndarray(shape = (len(test_image), img_size, img_size, 3),
                    dtype = np.float32)

for i in range(len(test_image)):
    x_test[i] = test_image[i]

x_test = x_test / 255

## Xây dựng và huấn luyện mô hình mạng CNN

Xây dựng mô hình

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization,Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

# Layer 1: Input  
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', input_shape=(img_size, img_size, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Layer 2  
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
   
# Layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Layer 4
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=256, kernel_size=(5, 5), padding='SAME', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Layer 5: Output
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Optimizing function: Adam
optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
model.summary()

Huấn luyện mô hình

In [ ]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Data augmentation for deep learning
datagen = ImageDataGenerator(rotation_range=360,      # Range for rotations
                             width_shift_range=0.2,   # Range for horizontal shifts
                             height_shift_range=0.2,  # Range for vertical shifts
                             zoom_range=0.2,          # Range for zoom
                             horizontal_flip=True,    # Range for horizontal flip
                             vertical_flip=True)      # Range for vertical flip

# With datagen, we have more data to get the best learning process
datagen.fit(x_train)

# Keras Callback: Important API to keep the best model while training
annealer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

# Training Model
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                              steps_per_epoch=x_train.shape[0] // 32,
                              epochs=120,
                              verbose=1,
                              callbacks=[annealer, checkpoint],
                              validation_data=(x_val, y_val))

## Đánh giá mô hình
Xem xét biểu đồ của Loss và Accuracy trong quá trình huấn luyện.

In [ ]:
h = history.history

plt.figure(1, figsize = (20, 6))

plt.subplot(121)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.plot(h['loss'], label = 'train')
plt.plot(h['val_loss'], label = 'validation')
plt.legend()

plt.subplot(122)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.plot(h['accuracy'], label = 'train')
plt.plot(h['val_accuracy'], label = 'validation')
plt.legend()

plt.show()

**Nhận xét**:
- Loss giảm từ khoảng 1.4 xuống còn khoảng 0.2.
- Accuracy tăng từ khoảng 0.3 lên khoảng 0.9.
Vậy, nhận xét chung là Loss giảm, và Accuracy tăng, do đó mô hình đã học hiệu quả.

## Dự đoán dữ liệu kiểm tra và lưu kết quả

Dự đoán dữ liệu kiểm tra

In [ ]:
y_test = model.predict(x_test)

predict = np.ndarray(shape = (x_test.shape[0], 4),
                     dtype = np.float32)

for i in range(0, y_test.shape[0]):
    for j in range(0, 4):
        if y_test[i][j] == max(y_test[i]):
            predict[i][j] = 1
        else:
            predict[i][j] = 0 

Lưu kết quả vào file

In [ ]:
healthy           = [y_test[0] for y_test in predict]
multiple_diseases = [y_test[1] for y_test in predict]
rust              = [y_test[2] for y_test in predict]
scab              = [y_test[3] for y_test in predict]

data = pd.DataFrame({'image_id'           : pd_test.image_id,
                     'healthy'            : healthy,
                     'multiple_diseases'  : multiple_diseases,
                     'rust'               : rust,
                     'scab'               : scab})

data.to_csv('submission.csv', index = False)

## Tổng kết
- Mạng neural tích chập (Convolution Neural Network - CNN) rất thích hợp để giải quyết vấn đề nhận diện bệnh trên lá cây.
- Kích thước ảnh được đưa vào mô hình có tác động mạnh đến độ chính xác và thời gian huấn luyện của mô hình.
  - img_size = 25: acc = 0.5562, thời gian huấn luyện = 148.8 giây.
  - img_size = 50: acc = 0.8729, thời gian huấn luyện = 288.7 giây.
  - img_size = 100: acc = 0.9256, thời gian huấn luyện = 619.1 giây.
- Hàm Adam giúp tối ưu hóa mô hình (img = 25, acc tăng từ 0.4501 lên 0.5562 khi sử dụng hàm tối ưu hóa Adam).



## Tham khảo
- HW02. Phân lớp bằng mạng Nơron: Đề bài và hướng dẫn thực hiện HW02, L.H.Thái, L.T.Phong, N.N.Thảo.
- Data Augmentation and Keras CNN: https://www.kaggle.com/code/nightwolfbrooks/data-augmentation-and-keras-cnn
- Plant Disease Classification Using Keras CNN: https://www.kaggle.com/code/seaceved/plant-disease-classification-using-keras-cnn
- Plant Pathology EDA and Deep CNN: https://www.kaggle.com/code/shawon10/plant-pathology-eda-and-deep-cnn

Cảm ơn mọi người đã đọc notebook này.